In [ ]:
# create docker image
!sudo nohup docker build -t llama3-deployment . &

In [ ]:
# create new repositroy in Amaon ECR
!aws ecr create-repository --repository-name your-repo-name --region eu-central-1

In [ ]:
# Get the login command from ECR and execute it directly
!aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin <your-account-id>.dkr.ecr.eu-central-1.amazonaws.com

In [ ]:
# Tag your Docker image and push it to ECR
!docker tag your-image-name:latest <your-account-id>.dkr.ecr.eu-central-1.amazonaws.com/your-repo-name:latest
!docker push <your-account-id>.dkr.ecr.eu-central-1.amazonaws.com/your-repo-name:latest

In [ ]:
# Use the AWS CLI to create the SageMaker model, specifying your custom image
!aws sagemaker create-model \
    --model-name your-model-name \
    --primary-container Image=<your-account-id>.dkr.ecr.eu-central-1.amazonaws.com/your-repo-name:latest \
    --execution-role-arn arn:aws:iam::your-account-id:role/your-sagemaker-execution-role \
    --region eu-central-1

In [ ]:
# Create the Endpoint Configuration Using AWS CLI
!aws sagemaker create-endpoint-config \
    --endpoint-config-name your-endpoint-config-name \
    --production-variants VariantName=AllTraffic,ModelName=your-model-name,InitialInstanceCount=1,InstanceType=ml.m5.xlarge,InitialVariantWeight=1.0 \
    --region eu-central-1

In [ ]:
# Create the Endpoint Using AWS CLI
!aws sagemaker create-endpoint \
    --endpoint-name your-endpoint-name \
    --endpoint-config-name your-endpoint-config-name \
    --region eu-central-1

In [ ]:
# Check the status of the endpoint
!aws sagemaker describe-endpoint --endpoint-name your-endpoint-name --region eu-central-1